<img width="50px" align="left" style="margin-right:20px" src="http://data.newprolab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.newprolab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 06 мая 2018, 23:59**

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

### Обработка данных на вход

Для выполнения работы вам следует взять файл http://data.newprolab.com/data-newprolab-com/project01/gender_age_dataset.txt и положить к себе в директорию `data`:

In [2]:
# First, we download the input data for the project:

data_dir = 'data'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir,filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

if not os.path.isdir(data_dir): os.mkdir(data_dir)
if not os.path.isfile(file_path):
    print('Downloading ' + filename + '...')
    urlretrieve(url, file_path)
    print('Download completed')

# Wait until you see that all files have been downloaded.
print('OK')

Download completed
OK


Он содержит данные о посещении сайтов ~40 000 пользователей, при этом по некоторым из них (~ 35 000) известны их пол и возрастная категория, а по 5 000 - эта информация не известна. В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

Первое, что обычно делают в таких случаях, — исследуют имеющийся датасет и разбираются, какие же данные мы получили.

Загрузим весь датасет в pandas:

In [3]:
df = pd.read_csv(file_path, sep='\t')

И теперь попробуем понять, что у нас есть:

In [4]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Что содержится в `user_json`?

In [5]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [12]:
json.loads(df.iloc[0].user_json)

{'visits': [{'timestamp': 1419688144068,
   'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun'},
  {'timestamp': 1426666298001,
   'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story'},
  {'timestamp': 1426666298000,
   'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html'},
  {'timestamp': 1426661722001,
   'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story'},
  {'timestamp': 1426661722000,
   'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html'}]}

Методом `pandas.DataFrame.apply` (хотя не только им) можно применить операцию десериализации json-строк ко всему датасету. Рекомендуем почитать [документацию по методу apply](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

Работая с подобными операциями, обратите внимание на kwargs-аргумент `axis`. Часто, забыв его указать, вы примените операцию не к ряду (строке), а к столбцу, что вряд ли входит в ваши планы.

### Очистка данных и feature engineering

Очистка данных и генерация новых фич составит значительную часть вашей работы. Именно здесь вы и должны продемонстрировать знания и креативность: чем лучше окажутся ваши фичи и чем лучше сможете убрать шум из датасета, тем лучших результатов вы достигнете.

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [13]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

Поскольку эта часть и есть ваша работа, мы не станем раскрывать все секреты (хотя несколько советов мы всё же дали, посмотрите ниже в разделе Подсказки).

### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [14]:
len(df[~((df.gender == '-') & (df.age == '-'))])

36138

Test set:

In [15]:
len(df[(df.gender == '-') & (df.age == '-')])

5000

In [16]:
len(df) # Весь датасет

41138

Когда вы очистили данные и сгенерировали признаки, которые можно дать на вход алгоритму, следующий этап — это разделить данные на тренировочную и тестовую выборки. Сохраните train и test выборки в отдельных файлах, используя метод `pandas.DataFrame.to_csv`. Либо просто сделайте два датафрейма: `train_df` и `test_df`. Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

### Сохранение модели

Обучать модель вы можете в ноутбуке - это удобно. А после того как модель обучилась и стала выдавать приемлемое качество на вашей валидационной выборке, сохраните ее в отдельном файле (например, используя pickle):

In [ ]:
#Сохранить модель, которая содержится в переменной vectorizer
import pickle
with open('TfidfVectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)


Однако, не забудьте сохранить и код генерации признаков и обучения модели - это нужно для воспроизводимости результатов.

### Обработка тестовых данных и формат вывода результатов

Помимо того, что у вас должна получиться точная модель, вам нужно уложиться в SLA (service-level agreement). Всё почти как по-настоящему. Результатом вашей работы в данном случае будет не выходной файл, в котором вы всё посчитали для скрытой выборки, а скрипт, который мы будем запускать и проверять SLA и точность.

Вот фрагмент кода, который считывает данные построчно из потока стандартного ввода:

In [19]:
data = pd.DataFrame(columns=['gender','age','uid','user_json'])
s=0
for line in sys.stdin:
    data.loc[s] = line.strip().split("\t")
    s+=1

Задача вашего скрипта сделать предсказание по всем полученным строкам и выдать результат в формате json. В файле должны быть только те пользователи, у которых пол и возрастная категория изначально неизвестны, и они должны быть **отсортированы по UID по возрастанию значений лексикографически.** Пример вывода указан ниже.

In [ ]:
output = output[['uid', 'gender', 'age']]
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
sys.stdout.write(output.to_json(orient='records'))

**Примечание**. Так как в данном проекте вам потребуется сохранять обученные модели в файле, то в вашем коде их нужно загружать явным образом. Иначе чекер не сможет запусть ваш скрипт корректно. Пример:

In [ ]:
#считать модель из файла в переменную vectorizer
import pickle
vectorizer= pickle.load(open('/data/home/test.user.bd8/TfidfVectorizer.pickle', 'rb'))

Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [ ]:
!tail -n1000 gender_age_dataset.txt | python3 project01_gender-age.py > output.json

### Подсказки

1. Есть много различных способов решить данную задачу: можно просто хорошо поработать с урлами и доменами, можно пропарсить содержимое этих урлов (заголовки, текст и т.д.) и воспользоваться неким векторизатором типа TF\*IDF для генерации дополнительных фич, которые уже в дальнейшем вы подадите на вход ML-алгоритму, можно сделать тематическое моделирование (LDA, BigARTM) сайтов и использовать одну или несколько тем в качестве фич.

2. Возможно, что данные грязные и их нужно дополнительно обработать. Спецсимволы, кириллические домены? Уделите этому этапу достаточно времени: здесь чистота датасета важнее, чем выбор алгоритма.

3. Часто бывает, что лучшее решение с точки зрения результата — оно же самое простое. Попробуйте сначала простые способы, простые алгоритмы, прежде чем переходить к тяжёлой артиллерии. Один из вариантов — начать с небольшого RandomForest.

4. Вам почти наверняка понадобится что-то из пакета sklearn. [Документация](http://scikit-learn.org/stable/user_guide.html) — ваш лучший друг.

5. Вы можете сначала предсказать пол, а затем возраст, либо сразу и то, и другое. Экспериментируйте.

6. Объединяйтесь в команды. Так гораздо веселее и интереснее.

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project1). До дедлайна вы будете проверять работу своего скрипта на валидационной выборке (2 000 записей). При наступлении дедлайна мы автоматически пересчитаем модели по скрытой тестовой выборке (3 000 записей). Это и будет финальным результатом.

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Важное замечание!** Вы должны дать прогноз хотя бы по 50% пользователей, у которых изначально не указан пол и возрастная категория. Иными словами, вы можете оставить неопределенными не более 50% изначально неопределенных пользователей.

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан, при условии что выполнен SLA в 0.04 секунды**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.

## Ваше решение здесь